In [3]:
# Script by Kevin Saavedra, Metro
# Adapted from Excel tables by Rich Arnold, P.E., ODOT

import os
import pandas as pd
import numpy as np
import datetime as dt

def TED_summation(df_teds):
    ########## Vehicle occupancy numbers
    VOCa = 1.4 
    VOCb = 10
    VOCt = 1
    ########## Working assumptions above
    df_teds['AVOc'] = df_teds['pct_auto'] * VOCa
    df_teds['AVOb'] = df_teds['pct_bus'] * VOCb
    df_teds['AVOt'] = df_teds['pct_truck'] * VOCt
    df_teds['TED'] = (df_teds['TED_seg'] * (df_teds['AVOc'] + df_teds['AVOb'] + df_teds['AVOt'])).round(3) 
    return df_teds

def total_excessive_delay(df_ted):
    df_ted['TED_seg'] = (df_ted['ED'] * df_ted['PK_HR']).round(2)
    df_ted['TED_seg'] = df_ted['TED_seg'].round() # Need this line to fix excel rounding weirdness

    #Groupby for summation by TMC
    ted_operations = ({'TED_seg' : 'sum',
                      'pct_auto' : 'max',
                      'pct_bus'  : 'max',
                      'pct_truck': 'max'})
    df_ted = df_ted.groupby('tmc_code', as_index=False).agg(ted_operations)
    return df_ted
    
def peak_hr(df_pk):
    df_pk['PK_HR'] = (df_pk['DirAADT_AUTO']/4).round()
    return df_pk
    
def excessive_delay(df_ed):
    df_ed['ED'] = df_ed['RSD']/3600 # check this value hundredths of an hour
    df_ed['ED'] = df_ed['ED'].round(3)
    df_ed['ED'] = np.where(df_ed['ED'] >= 0, df_ed['ED'], 0)
    return df_ed

def RSD(df_rsd):
    # returns travel time segment delay calculations.
    df_rsd['RSD'] = df_rsd['SD'] - df_rsd['2017_EDT']
    df_rsd['RSD'] = np.where(df_rsd['RSD'] >= 0, df_rsd['RSD'], 0)
    return df_rsd
        
def segment_delay(df_sd):
    # returns SD spreadsheet value.
    df_sd['SD'] = df_sd['travel_time_seconds'].round()
    return df_sd

def AADT_splits(df_spl):
    # Apply mode splits using ODOT methodology
    df_spl['TOTAL_AADT'] = df_spl['DirAADT_AUTO'] + df_spl['DirAADT_BUS'] + df_spl['DirAADT_TRK']
    df_spl['pct_auto'] = df_spl['DirAADT_AUTO']/df_spl['TOTAL_AADT'] 
    df_spl['pct_bus'] = df_spl['DirAADT_BUS']/df_spl['TOTAL_AADT'] 
    df_spl['pct_truck'] = df_spl['DirAADT_TRK']/df_spl['TOTAL_AADT'] 
    return df_spl

def main():
    pd.set_option('display.max_rows', None)

    df = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/Feb2017_test.csv')) #fix in script implementation
    # Filter by timestamps
    df['measurement_tstamp'] = pd.to_datetime(df['measurement_tstamp'])
    df = df[df['measurement_tstamp'].dt.weekday.isin([0, 1, 2, 3, 4])] # Capture weekdays only
    df = df[df['measurement_tstamp'].dt.hour.isin([6, 7, 8, 9, 16, 17, 18, 19])]
    # Join relevant files
    df_meta = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/TMC_Identification.csv'), 
                          usecols=['tmc', 'miles', 'tmclinear', 'aadt', 'aadt_singl', 'aadt_combi' ])
    df_odot = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/odot_edt.csv'))    
    df = pd.merge(df, df_meta, left_on=df['tmc_code'], right_on=df_meta['tmc'], how='inner')
    df = pd.merge(df, df_odot, left_on=df['tmc_code'], right_on=df_odot['TMC'], how='inner')
    # Apply calculation functions
    df = AADT_splits(df)
    df = segment_delay(df)
    df = RSD(df)
    df = excessive_delay(df)
    df = peak_hr(df)
    df = total_excessive_delay(df)
    df = TED_summation(df)
    #print(df)                                       
    # Return a selected row for test purposes only
    #df = df[['aadt','DirAADT_AUTO','RSD','PK_HR','ED', 'tmc_code', 'TED']]
    df = df[['tmc_code','TED']]
    print(df)
    #print(df.loc[df['tmc_code'] == '114-04369'])

In [4]:
main()

       tmc_code         TED
0     114+04369   15490.188
1     114+04370     863.263
2     114+04371    2280.380
3     114+04372     469.349
4     114+04373    2880.899
5     114+04374    1263.976
6     114+04375    6755.910
7     114+04376   28254.661
8     114+04377    4902.573
9     114+04378    1742.269
10    114+04379   34246.438
11    114+04380    7865.178
12    114+04381     992.499
13    114+04385   94010.311
14    114+04386  130638.581
15    114+04387   27326.875
16    114+04388     505.815
17    114+04389      90.616
18    114+04390    2309.471
19    114+04391     418.606
20    114+04392     197.709
21    114+04393    1856.784
22    114+04394   69227.541
23    114+04395  159704.595
24    114+04396   91909.868
25    114+04397   20990.989
26    114+04398   52745.241
27    114+04399     237.629
28    114+04400   19801.864
29    114+04401  160847.682
30    114+04402   80212.465
31    114+04403   20354.003
32    114+04404   11370.638
33    114+04405   43266.924
34    114+04411  207